## Чат бот на 2 датасета с распознаванием текста, лиц и голосовых сообщений

In [1]:
# импортируем библиотеки
import numpy as np
import pandas as pd
import cv2
from io import BytesIO
from imutils import paths
import face_recognition
import pickle
import os
import face_recognition
import imutils
import pickle
import time
import cv2
import os

import logging 

import pytesseract
import speech_recognition as sr
import subprocess

from urllib.request import urlopen

### Загрузка 1-го датасета

In [2]:
# загрузим 1-й датасет в переменную

with open('dialogues.txt', encoding='utf-8') as f:
    content = f.read()

# считываем датасет, разделяя его на диалоги двойным переносом строки

dialogues = [dialogue_line.split('\n') for dialogue_line in content.split('\n\n')]

# пишем функцию, которая приводит запрос пользователя к единому виду, удобному для обработки моделью
# выполняем очистку введенного текста запроса

def filter_text(text):
    text = text.lower()
    text = [c for c in text if c in 'abcdefghijklmnopqrstuvwxwzабвгдежзийклмнопрстуфхцчшщъыбэюя- '] #abcdefghijklmnopqrstuvwxwz 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя- '
    text = ''.join(text)
    return text

questions = set() # функция убирающая повторы из вопросов
qa_dataset = [] # [[q, a], ...]

for replicas in dialogues: 
    if len(replicas) < 2:  # если длина реплики меньше 2х (нет вопроса и ответа), то не берем эту реплику в наш датасет и переходим к следующей
        continue 
        
    question, answer = replicas[:2] #выделяем вопрос и ответ из реплики
    question = filter_text(question[2:]) # отрезаем пару ненужнх символов в начале
    answer = answer[2:] # отрезаем пару ненужнх символов в начале
    
    if question and question not in questions: # если question не пустой и если он уже не встречался ранее в списке вопросов (question), тогда мы его добавляем в наш датасет и в список question
        questions.add(question) # добавляем новый question в список questions
        qa_dataset.append([question, answer]) # добавляем извлеченные вопросы и ответы из начального "сырого" датасета в наш датасет вопросов-ответов
    
# создадим упрощенный датасет-словарь для ускорения, струткуру поиска 
qa_by_word_dataset = {} #{'word': [[q, a], ...]} 
# создаем словарь, где ключом является слово, которое встречается во фразе,заданной пользователем, 
# а значением является список из вопросов и ответов (мини дата сет),в которых встречается это слово 
# 
# пройдемся по всем репликам датасета qa,разобъем их на слова и каждому слову будет соотвестствовать список реплик (пар вопрос-ответ), где встречается это слово

for question, answer in qa_dataset:
    words = question.split(' ')
    for word in words: # для каждого слова в заданном вопросе
        if word not in qa_by_word_dataset: # в случае отсутствия слов в словаре qa_by_word_dataset создаем список
            qa_by_word_dataset[word] = []
        qa_by_word_dataset[word].append((question, answer)) # добавляем в датасет пару по ключевому слову


In [3]:
dialogues[0:3]

[['- Пока, толстуха!', '- Пока, малышка!'],
 ['- Этому надо положить конец,', '- Это не жизнь!'],
 ['- А ты не помолчишь?', '- Замолчу, когда захочу.']]

In [4]:
qa_dataset[0:3]

[['пока толстуха', 'Пока, малышка!'],
 ['этому надо положит конец', 'Это не жизнь!'],
 ['а ты не помолчиш', 'Замолчу, когда захочу.']]

In [5]:
# преобразуем список списков в датасет
df = pd.DataFrame(qa_dataset, columns =['context_0', 'reply'])

In [6]:
df.head(3)

,context_0,reply
0,пока толстуха,"Пока, малышка!"
1,этому надо положит конец,Это не жизнь!
2,а ты не помолчиш,"Замолчу, когда захочу."


### Загрузка 2-го датасета

In [7]:
# загружаем датасет
good1 = pd.read_csv('good.tsv', sep = '\t')

In [8]:
good1.head(3)

,context_2,context_1,context_0,reply,label
0,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",не могу .,good
1,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .","нет , звонить буду я .",neutral
2,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",я не прекращу звонить .,good


In [9]:
# извлекаем пару столбцов  "вопрос-ответ" из датасета
good2=good1.loc[:,['context_0', 'reply']]

### объединяем датасеты 1 и 2 в общий датасет

In [10]:
# объединяем датасеты 1 и 2 в общий датасет
good = pd.concat([good2, df], ignore_index = True)

In [11]:
good

,context_0,reply
0,"ладно , повесь трубку .",не могу .
1,"ладно , повесь трубку .","нет , звонить буду я ."
2,"ладно , повесь трубку .",я не прекращу звонить .
3,"ладно , повесь трубку .",я звоню им .
4,"я пытаюсь добраться до юджина , но , кажется ,...","куда же вы едете , месье ?"
...,...,...
664783,а тепер ступай туда за королевой и мы доставим...,Что там?
664784,вот выпей тебе станет легче нет нет пока ни слова,Полегчало?
664785,но почему ты спрашиваеш меня я думала что,Что ты думала?
664786,никак потерял что,Да вот сокровище пропало.


### Векторизация

In [12]:
# векторизация текстов
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer #импортируем разные типы векторайзеров
# создаем объект, который будет преобразовывать в числовые векторы
vectorizer = TfidfVectorizer() # 
# обучаем его на всех контекстах -> запоминаем частоту каждого слова
vectorizer.fit(good.context_0) 
# записываем в матрицу, сколько раз каждое слово встречалось в каждом тексте
matrix_big = vectorizer.transform(good.context_0)
print(matrix_big.shape) #размер матрицы = число фраз * размер словаря

(664788, 195496)


### Трансформирование модели

In [13]:
# сокращение размерности, для ускорения модели
# импортируем метод главных компонент
from sklearn.decomposition import TruncatedSVD
# проецируем данные в 900 мерное пространство
svd = TruncatedSVD(n_components = 900)
svd.fit(matrix_big)
matrix_small = svd.transform(matrix_big)
# количество строк(наблюденияй) * количество столбцов
print(matrix_small.shape)
# % сохранения исходной информаации
print(svd.explained_variance_ratio_.sum())

(664788, 900)
0.37434922438700263


### Модель обработки текста способом ближайших соседей

In [14]:
from sklearn.neighbors import BallTree
from sklearn.base import BaseEstimator

def softmax(x):
    # функция для создания вероятностного распределения
    proba = np.exp(-x)
    return proba / sum(proba)

class NeighborSampler(BaseEstimator):
    # Класс для случайного выбора одного из ближних соседей
    def __init__(self, k=5, temperature=1.0):
        self.k = k
        self.temperature = temperature
    def fit(self, X, y):
        self.tree_ = BallTree(X)
        self.y_ = np.array(y)
    def predict(self, X, random_state=None):
        distances, indices = self.tree_.query(X, return_distance=True, k=self.k)
        result = []
        for distance, index in zip(distances, indices):
            result.append(np.random.choice(index, p=softmax(distance * self.temperature)))
            return self.y_[result]

### Объединение всех моделей

In [15]:
# соединяем векторизацию, сокращение размерности и поиск соседей
from sklearn.pipeline import make_pipeline
ns = NeighborSampler()
ns.fit(matrix_small, good.reply)
pipe = make_pipeline(vectorizer, svd, ns) 

### Прикрутим модель обработки текста к Telegramm и добавим туда распознавание текста, конвертирование аудиосообщения в текст и детектирование лиц на изображениях

In [16]:
# в директории Images хранятся папки со всеми изображениями и идентификаторами
imagePaths = list(paths.list_images('Images')) 
knownEncodings = []
knownNames = []
# перебираем все папки с изображениями
for (i, imagePath) in enumerate(imagePaths):
    # извлекаем идентификатор из названия папки
    name = imagePath.split(os.path.sep)[-2]
    # загружаем изображение и конвертируем его из BGR в dlib ordering (RGB)
    image = cv2.imread(imagePath) 
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #используем библиотеку Face_recognition для обнаружения лиц
    boxes = face_recognition.face_locations(rgb,model='hog')
    # вычисляем эмбеддинги для каждого лица
    encodings = face_recognition.face_encodings(rgb, boxes)
    # loop over the encodings
    for encoding in encodings:
        knownEncodings.append(encoding)
        knownNames.append(name)
# сохраним эмбеддинги вместе с их именами в формате словаря
data = {"encodings": knownEncodings, "names": knownNames}
#print('data', data)
# для сохранения данных в файл используем метод pickle
f = open("face_enc", "wb") #f = open("face_enc", "wb")
f.write(pickle.dumps(data))
f.close()

In [27]:
# прикрутим модель бота к Telegramm
from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)

updater = Updater("1603824366:AAET-yxymYaKaMXjjmq4HCb__JT08t5czCQ")
# Define a few command handlers. These usually take the two arguments update and
# context.
def start(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_text('1. При отправке текстового сообщения бот отправит Вам на него ответное сообщение. 2. При отправке изображения: Если бот распознает на нем лицо - он обведет его в квадрат и подпишет идентификатором. Если бот найдет там текст - он выведет его в виде сообщения. 3. При отправке голосового сообщения: бот пришлет Вам его в текстовом виде.')

# функция обработки не распознных команд
def unknown(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, 
                             text="Sorry, I didn't understand that command. Неизвестная команда, наберите /start")    

def help_command(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('Набери /start!')


def echo(update: Update, context: CallbackContext) -> None:
    """Echo the user message."""
    answer = pipe.predict([update.message.text.lower()])[0] # обработка ответа
    update.message.reply_text(answer) 
    #print(stats)
    # печать ника отправителя сообщения и текста запроса сообщения
    print('«' + update.message.chat.first_name + '»' + ': -' + update.message.text) 
    print('Бот: -', answer) # печать ответа

# функция распознавания текста и лиц
def handle(update: Update, context: CallbackContext) -> None:
    
    #путь для подключения tesseract
    pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'    
    # путь к XML-файлу, содержащему каскадный файл haar 
    cascPathface = os.path.dirname(cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
      
    # загрузим haarcascade в каскадный классификатор
    faceCascade = cv2.CascadeClassifier(cascPathface)
    
    # загрузим сохраненные известные эмбеддинги 
    data = pickle.loads(open("face_enc", "rb").read())
    
    #возьмем фото из сообщения
    fileID = update.message.photo[-1] 
    file = context.bot.get_file(fileID)
    
    #загружаем в переменную прочитанный файл
    down_file = urlopen(file.file_path).read()
    
    # записывем загруженный боту файл локально
    with open("image3.jpg", 'wb') as f:  
        f.write(down_file)
        
    # Подключение фото
    img3 = cv2.imread("image3.jpg")
    rgb = cv2.cvtColor(img3, cv2.COLOR_BGR2RGB)
    #преобразуем фото в оттенки серого
    gray = cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY)
    
    faces = faceCascade.detectMultiScale(gray,
                                     scaleFactor=1.1,
                                     minNeighbors=5,
                                     minSize=(60, 60),
                                     flags=cv2.CASCADE_SCALE_IMAGE)

    # извлекаем эмбеддинги из присланного фото
    encodings = face_recognition.face_encodings(rgb)
    names = []
   
    for encoding in encodings:
        # Сравним эмбеддинги из присланного фото с эмбеддинги в данных ["кодировки"] 
        # Совпадения содержат массив с логическими значениями и True для вложений, которым он точно соответствует и False при отсутствии
        matches = face_recognition.compare_faces(data["encodings"], encoding)
        # имя набора = Unknown, если кодировка не совпадает
        name = "Unknown"
        #проверим, наличие совпадений
        if True in matches:
            #найдем позиции, в которых мы получаем True, и сохраним их 
            matchedIdxs = [i for (i, b) in enumerate(matches) if b]
            counts = {}
            #цикл на сопоставленных эмбеддингах и подсчет для каждого распознанного лица 
            for i in matchedIdxs:
                #проверяем идентификаторы по соответствующим эмбеддингам, которые мы сохранили в matchedIdxs 
                name = data["names"][i]
                counts[name] = counts.get(name, 0) + 1
                name = max(counts, key=counts.get)
            # обновление списка идентификаторов 
            names.append(name)
            # цикл по распознанным лицам
            for ((x, y, w, h), name) in zip(faces, names):
                # изменение масштаба координат лица
                img3 = cv2.rectangle(img3, (x, y), (x + w, y + h), (0, 255, 0), 2)
                # подпись идентификатора распознанного лица на изображении 
                txt3 = cv2.putText(img3, name, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1.85, (0, 255, 0), 2)
    
    #выводим текст с картинки
    config = r'--oem 3 --psm 6'
    
    # печатаем ответ локально
    print("Распознан текст: «", pytesseract.image_to_string(img3, config=config, lang='rus+eng') + '»') #для русского и английского
    answer1 = pytesseract.image_to_string(img3, config=config, lang='rus+eng') #для русского и английского 
    
    cv2.imwrite('image3.jpg', img3)
    img3 = BytesIO(open("image3.jpg", "rb").read())

    # отправляем детектированное лицо с картинки  
    update.message.reply_photo(photo=img3)
    # отправляем распознанный текст с картинки
    update.message.reply_text("Распознан текст: «" + answer1 + '»')

    
# функция распознавания преобразования голосового сообщения в текст    
def voice(update: Update, context: CallbackContext) -> None:
    # для распознавания текста нам потребуется класс Recognizer он имеет множество функций, а также определяет каким API мы будем пользоваться
    r = sr.Recognizer()
    #получаем оцифрованную звуковую дорожку и записываем ее в файл.
    fileID = update.message.voice
    file = context.bot.get_file(fileID)
    down_file = urlopen(file.file_path).read()
    with open("audio.ogg", "wb") as fot:
        fot.write(down_file)
    # конвертируем файл из формата .ogg в формат .wav    
    process = subprocess.run(['ffmpeg', '-i', 'audio.ogg', 'audio.wav', '-y']) 
    # открываем файл и считываем его содержимое, сохраняя данные в экземпляре AudioFile
    file1 = sr.AudioFile('audio.wav')
    with file1 as source:
        try:#для дальнейшей работы требуется преобразовать сигнал в объект модуля Speech_recognition для этого существует метод record()
            audio = r.record(source)
            #для расшифровки сигнала используется метод recognize_google()
            #выбранный Api поддерживает русский язык
            text = r.recognize_google(audio, language='ru-RU`')
            update.message.reply_text(text)
            print('аудио: ', text)
        except sr.UnknownValueError:
            update.message.reply_text("Произошла ошибка при распознавании голоса!")
            print("Произошла ошибка при распознавании голоса!")
        
def main() -> None:
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    # создаем объект Updater
    updater = Updater("1603824366:AAET-yxymYaKaMXjjmq4HCb__JT08t5czCQ")
    # cоздаем объект dispatcher
    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher
    # обработчики встроенных запросов 
    # on different commands - answer in Telegram
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))

    # on non command i.e message - echo the message on Telegram
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))
    # обработчик графических сообщений типа photo  
    dispatcher.add_handler(MessageHandler(Filters.photo & ~Filters.text , handle))
    # обработчик голосовых сообщений типа voice  
    dispatcher.add_handler(MessageHandler(Filters.voice & ~Filters.text , voice))
    # обработчик не распознных команд
    unknown_handler = MessageHandler(Filters.command, unknown)
    dispatcher.add_handler(unknown_handler)
    
    # Start the Bot
    # запуск прослушивания сообщений
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    # обработчик нажатия Ctrl+C
    updater.idle()


if __name__ == '__main__':
    main()

2021-11-15 19:54:58,863 - apscheduler.scheduler - INFO - Scheduler started


«Alex»: -Привет
Бот: - здравствуй .
«Alex»: -Разгоняйся
Бот: - Ну долго нам еще?
«Alex»: -Надеюсь, что сегодня
Бот: - Нет, сегодня не церковный праздник.
«Alex»: -Вроде бы, но мы к этому не привязаны
Бот: - И на кого же, господин Маркварт?
«Alex»: -Хз
Бот: - 200р.
«Alex»: -Дороговато
Бот: - Бабник, У него даже взгляд неприятный.
«Alex»: -😁
Бот: - что такое , мини ?
«Alex»: -Аусвайс
Бот: - Ванная комната.
Распознан текст: « ХОРОШО, ЧТО ПЯТНИЦА
УЖЕ СЕГОДНЯ,
А НЕ ЗАВТРА, КАК ВЧЕРА.
»


2021-11-16 00:35:09,009 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...
2021-11-16 00:35:09,071 - apscheduler.scheduler - INFO - Scheduler has been shut down
